# (G12) 번역가는 대화에도 능하다
****
# Project: 멋진 챗봇 만들기

## Step 1. 데이터 로드
```c
$ git clone https://github.com/songys/Chatbot_data.git
$ mv Chatbot_data ~/aiffel/transformer_chatbot 
```

In [2]:
# 필요한 라이브러리 선언
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.1


In [3]:
df = pd.read_csv("/home/aiffel-dj20/aiffel/transformer_chatbot/Chatbot_data/ChatbotData .csv")
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [4]:
questions = df['Q'].values.tolist()
answers = df['A'].values.tolist()

## Step 2. 데이터 정제

- ```preprocess_sentence()``` 함수
> 1. 영문자는 모두 소문자 변환.
> 2. 영문자와 한글, 숫자 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거.

In [8]:
def preprocess_sentence(sentence):
    new_sen = []    
    for i in sentence:
        i = i.lower()
        i = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 a-z A-Z?.!,0-9\\s]+", "", i)
        new_sen.append(i)
    return new_sen
print("done")

done


## Step 3. 데이터 토큰화

- 토큰화에는 KoNLPy의 ```mecab```클래스를 사용.
- 아래 조건을 만족하는 ```build_corpus()```함수를 구현.
> 1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받음.
> 2. 데이터를 앞서 정의한 ```preprocess_sentence()```함수로 정제하고, 토큰화.
> 3. 토큰화는 전달받은 토크나이즈 함수를 사용. 이번엔 mecab.morphs 함수를 전달.
> 4. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외.
> 5. 중복되는 문장은 데이터에서 제외합니다. ```소스 : 타겟``` 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사. 중복 쌍이 흐트러지지 않도록 유의.


- 구현한 함수를 활용하여 ```questions```와 ```answers```를 각각 ```que_corpus```, ```ans_corpus```에 토큰화하여 저장.

In [9]:
from konlpy.tag import Mecab

def build_corpus(src, tgt):
    tokenizer = Mecab()
     
    src_ = preprocess_sentence(src)
    tgt_ = preprocess_sentence(tgt)
    
    que_corpus = []
    ans_corpus = []
    
    for i in src_:
        que_corpus.append(tokenizer.morphs(i))
    
    for i in tgt_: 
        ans_corpus.append(tokenizer.morphs(i))
    
    return que_corpus, ans_corpus

print("done")

done


In [10]:
que_corpus, ans_corpus = build_corpus(questions, answers)

print(len(que_corpus))
print(len(ans_corpus))

11823
11823


In [11]:
print(que_corpus[:5])

[['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'], ['ppl', '심하', '네']]


In [12]:
print(ans_corpus[:5])

[['하루', '가', '또', '가', '네요', '.'], ['위로', '해', '드립니다', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['눈살', '이', '찌푸려', '지', '죠', '.']]


## Step 4. Augmentation

- Lexical Substitution을 실제로 적용해.
```c
https://drive.google.com/u/0/ucid=0B0ZXk88koS2KbDhXdWg1Q2RydlU&export=download 
```


- 다운로드한 모델을 활용해 데이터를 Augmentation. 앞서 정의한 ```lexical_sub()```함수를 참고.
- Augmentation된 ```que_corpus```와 원본```ans_corpus```가 병렬을 이루도록, 이후엔 반대로 원본```que_corpus```와 Augmentation된```ans_corpus```가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 함.

In [13]:
import gensim

wv = gensim.models.Word2Vec.load('/home/aiffel-dj20/aiffel/transformer_chatbot/Chatbot_data/ko/ko.bin')

In [14]:
# Lexical Substitution 구현하기
def lexical_sub(sentence, word2vec):
    import random

    res = []
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res.append(_to)
        else: res.append(tok)

    return res

In [15]:
src_temp = []
tgt_temp = []

que_arg = []
ans_arg = []

In [16]:
for i in range(len(que_corpus)):
    que_temp = lexical_sub(que_corpus[i], wv)
    ans_temp = lexical_sub(ans_corpus[i], wv)
    que_arg.append(que_temp)
    ans_arg.append(ans_temp)

/home/aiffel-dj20/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


In [17]:
for i in range(20):
    if que_corpus[i] != que_arg[i]:
        print(que_corpus[i],que_arg[i])
        print(i)

['12', '시', '땡', '!'] ['12', '시가', '땡', '!']
0
['1', '지망', '학교', '떨어졌', '어'] ['1', '지망', '학교', '떨어졌', '어서']
1
['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'] ['3', '박', '4', '월과', '놀', '러', '가', '고', '싶', '다']
2
['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'] ['3', '박', '4', '월과', '정도', '놀', '러', '가', '고', '싶', '다']
3
['ppl', '심하', '네'] ['ppl', '심하', '카나']
4
['sd', '카드', '망가졌', '어'] None
5
['sd', '카드', '안', '돼'] None
6
['sns', '맞', '팔', '왜', '안', '하', '지', 'ㅠㅠ'] ['sns', '맞', '팔', '과연', '안', '하', '지', 'ㅠㅠ']
7
['sns', '시간', '낭비', '인', '거', '아', '는데', '매일', '하', '는', '중'] ['sns', '시간', '낭비', '인', '거', '아', '으며', '매일', '하', '는', '중']
8
['sns', '시간', '낭비', '인데', '자꾸', '보', '게', '됨'] None
9
['sns', '보', '면', '나', '만', '빼', '고', '다', '행복', '해', '보여'] ['sns', '보', '면', '나의', '만', '빼', '고', '다', '행복', '해', '보여']
10
['가끔', '궁금', '해'] ['이따금', '궁금', '해']
11
['가끔', '뭐', '하', '는지', '궁금', '해'] ['가끔', '뭐', '하', '는지', '궁금하', '해']
12
['가끔', '은', '혼자', '인', '게', '좋', '다'] ['가끔', '은', '거기', '인

In [18]:
src_temp = que_arg    + que_corpus + que_corpus
tgt_temp = ans_corpus + ans_arg    + ans_corpus

In [19]:
que_corpus = src_temp
ans_corpus = tgt_temp

In [20]:
print(len(que_corpus))
print(len(ans_corpus))

35469
35469


## Step 5. 데이터 벡터화

- 타겟 데이터인 ```ans_corpus```에 ```<start>```토큰과 ```<end>```토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행. 
- 우리가 구축한 ```ans_corpus```는 list 형태이기 때문에 아주 쉽게 이를 해결함.

In [21]:
for i in range(len(que_corpus)):
    que_corpus[i] = ['<start>'] + que_corpus[i] + ['<end>']

TypeError: can only concatenate list (not "NoneType") to list

****
# 루브릭 평가

|평가문항|상세기준|
|:----|:----|
|1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?|챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.|
|2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?|과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.|
|3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?|주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.|

1. 1번은 길이 조정과 벡터화를 제외하곤 주어진 조건에 맞춰서 최대한 결과를 도출하려고 노력했다. 그런데 list라서 lower이 안된다는 문제를 가지고 오랫동안 고민하는 바람에 다른 것을 풀 시간이 없었다. for문을 쓰면 되는 거였다는 걸 깨닫는데는 많은 시간이 걸렸다.


2. 트랜스포머 모델은 구현하지 못했다.


3. 챗봇을 만들지 못했다.

# 회고

챗봇을 만들어야 하는데 솔직히 말하면 요즘 노드에 손이 잘 가지 않는다. 그래서 덮어두고 보지 않고 있다. 챗봇을 한다고 말을 했지만 잘 모르겠다. 잘 모르는 상태에서 어려운 노드를 보니 고통스럽고, 그 고통에서 벗어나기 위해 공부를 멀리하고 있는 것 같다. 이 악순환의 고리를 끊어야 할텐데... 아무것도 하기 싫다.